In [ ]:
%tensorflow_version 1.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

!pip install -q keras
import keras

!pip install gensim

!pip install nltk
import nltk

TensorFlow 1.x selected.
Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [ ]:
import numpy as np
import pandas as pd

nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
import timeit

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
first_fold=pd.read_csv('/content/drive/My Drive/input/1-Fold.csv')
second_fold=pd.read_csv('/content/drive/My Drive/input/2-Fold.csv')
thrid_fold=pd.read_csv('/content/drive/My Drive/input/3-Fold.csv')
fourth_fold=pd.read_csv('/content/drive/My Drive/input/4-Fold.csv')
fifth_fold=pd.read_csv('/content/drive/My Drive/input/5-Fold.csv')

frames=[second_fold,thrid_fold,fifth_fold,fourth_fold]
data = pd.concat(frames, ignore_index=True) #concate 8000


In [ ]:
train_data=data
test_data= first_fold
# test_data= second_fold
# test_data= thrid_fold
# test_data= fourth_fold
# test_data= fifth_fold

In [ ]:
star=train_data.stars.unique()
dic={}
print(star)
for i,stars in enumerate(star):
    dic[stars]=i
labels=train_data.stars.apply(lambda x:dic[x])
# print(labels)

[3 4 5 2 1]


In [ ]:
val_data=train_data.sample(frac=0.2499849588,random_state=200)#valdata 
train_data=train_data.drop(val_data.index)
print(len(val_data))
print(len(train_data))

2000
6000


In [ ]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

6000
2000
2000


In [ ]:
reviews=train_data.text

In [ ]:

rev=[] #remove punctuation
for i in range(len(reviews)):
  review=reviews.iloc[i]
  rev.append(review.translate(str.maketrans('', '', string.punctuation)))

stopwords = nltk.corpus.stopwords.words('english') #set and add stopwords list
tokens_without_sw=[] #Stopords removal
for i in range(len(rev)):
  tokenized_word=word_tokenize(rev[i])
  tokens_lw=[word.lower() for word in tokenized_word]
  tokens_sw =[word for word in tokens_lw if not word in stopwords]
  filtered_sentence = (" ").join(tokens_sw)
  tokens_without_sw.append(filtered_sentence)
train_data=train_data.assign(text=tokens_without_sw)

In [ ]:
train_data['text']=train_data['text'].astype('str') 
test_data['text']=test_data['text'].astype('str')
val_data['text']=val_data['text'].astype('str')
reviews=train_data.text

In [ ]:
reviews=train_data.text

In [ ]:
#Process Tokenizer
NUM_WORDS=2000 
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(reviews)
sequences_train = tokenizer.texts_to_sequences(reviews)
sequences_valid=tokenizer.texts_to_sequences(val_data.text)
sequences_test=tokenizer.texts_to_sequences(test_data.text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 27701 unique tokens.


In [ ]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_train = to_categorical(np.asarray(labels[train_data.index]))
y_val = to_categorical(np.asarray(labels[val_data.index]))

print('Shape of X train, X validation, X test tensor:', X_train.shape,X_val.shape,X_test.shape)
print('Shape of label train, validation, test tensor:', y_train.shape,y_val.shape)

Shape of X train, X validation, X test tensor: (6000, 425) (2000, 425) (2000, 425)
Shape of label train, validation, test tensor: (6000, 5) (2000, 5)


In [ ]:


word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/input/GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

In [ ]:
start = timeit.default_timer()
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)


maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)


merged_tensor = concatenate([maxpool_0,maxpool_1,maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(reshape)
output = Dense(units=5, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

model = Model(inputs, output)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [ ]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 425)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 425, 300)     600000      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 425, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 423, 1, 100)  90100       reshape_1[0][0]                  
____________________________________________________________________________________________

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss')]
history = model.fit(X_train, y_train, batch_size=50, epochs=20, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks)



Train on 6000 samples, validate on 2000 samples
Epoch 1/50
6000/6000 [==============================] - 16s 3ms/step - loss: 1.5589 - acc: 0.4042 - val_loss: 1.4691 - val_acc: 0.4485
Epoch 2/50
6000/6000 [==============================] - 10s 2ms/step - loss: 1.4041 - acc: 0.4953 - val_loss: 1.3748 - val_acc: 0.4920
Epoch 3/50
6000/6000 [==============================] - 10s 2ms/step - loss: 1.3047 - acc: 0.5385 - val_loss: 1.3481 - val_acc: 0.5010
Epoch 4/50
6000/6000 [==============================] - 10s 2ms/step - loss: 1.2572 - acc: 0.5657 - val_loss: 1.3214 - val_acc: 0.5325
Epoch 5/50
6000/6000 [==============================] - 10s 2ms/step - loss: 1.2181 - acc: 0.5807 - val_loss: 1.3534 - val_acc: 0.5050


In [ ]:
del(sequences_train)
del(sequences_valid)
del(reviews)

In [ ]:
y_pred=model.predict(X_test)
stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  59.930272081999874


In [ ]:
to_submit=pd.DataFrame(index=test_data.review_id,data={'1':y_pred[:,dic[1]],
                                                '2':y_pred[:,dic[2]],
                                                '3':y_pred[:,dic[3]],
                                                '4':y_pred[:,dic[4]],
                                                '5':y_pred[:,dic[5]],
                                                })

In [ ]:
#pd.DataFrame(to_submit)
prediction=[]
for i in range(y_pred.shape[0]):
  rating_pred=np.argmax(y_pred[i])
  if rating_pred == dic[1]:
    rating_pred=1
  elif rating_pred == dic[2]:
    rating_pred=2 
  elif rating_pred == dic[3]:
    rating_pred=3
  elif rating_pred == dic[4]:
    rating_pred=4
  else :
    rating_pred=5
  prediction.append(rating_pred)


In [ ]:
to_submit.to_csv('hasil1_softmax.csv')

In [ ]:
real= np.array(test_data.stars)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(real,prediction)
print(accuracy)

0.5035


In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(real, prediction) #Perhitungan F1 score dengan kelas {1,2,3,4,5}
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :', accuracy_score(real, prediction)) 
print ('Report : ')
print (classification_report(real, prediction))

Confusion Matrix :
[[ 51  21  15  49  15]
 [ 22  22  54  77   7]
 [  6   7  66 201  12]
 [  6   1  17 573 101]
 [  4   2   6 370 295]]
Accuracy Score : 0.5035
Report : 
              precision    recall  f1-score   support

           1       0.57      0.34      0.42       151
           2       0.42      0.12      0.19       182
           3       0.42      0.23      0.29       292
           4       0.45      0.82      0.58       698
           5       0.69      0.44      0.53       677

    accuracy                           0.50      2000
   macro avg       0.51      0.39      0.40      2000
weighted avg       0.53      0.50      0.48      2000



In [ ]:
import math
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(real, prediction)
print(mse)
rmse=math.sqrt(mse)
print(rmse)

1.0705
1.0346496991735898
